In [1]:
import numpy as np
import pandas as pd
import gcsfs
from tqdm import tqdm
tqdm.pandas()
from google.cloud import bigquery
client = bigquery.Client()
from scipy.sparse import coo_matrix

# Make the data

In [316]:
# ContentMetadataView data
query = """
    CREATE TEMP FUNCTION strip_str_array(val ANY TYPE) AS ((
      SELECT ARRAY_AGG(DISTINCT TRIM(t))
      FROM UNNEST(val) t
      WHERE t != ""
    ));

    WITH source AS (
        SELECT ANY_VALUE(TitleDetails_title) AS title, TitleType AS type, 
            STRING_AGG(TitleDetails_LongSynopsis, " ") AS synopsis,
            LANGUAGE AS language,
            STRING_AGG(ARRAY_TO_STRING([TitleGenre, TitleSubgenres, TitleTags, LANGUAGE], ","), ",") AS tags
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView`
        WHERE LANGUAGE IN ("eng", "spa")
        AND TitleType IN ("Programme", "Movie")
        AND TitleDetails_title NOT IN ("OnDemand Movie")
        GROUP BY LOWER(TitleDetails_title), TitleType, LANGUAGE
    ),

    test_data AS (
        SELECT title, type, language, synopsis, 
           strip_str_array(SPLIT(tags, ",")) AS tags
        FROM source
    )

    SELECT * FROM test_data
"""
table_id = f"{client.project}.metadata_enhancement.synopsis_cmv_full_tags_edc_dev"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

client.query(query=query, job_config=job_config,  location="US").result()

In [ ]:
%%bigquery

    CREATE TEMP FUNCTION strip_str_array(val ANY TYPE) AS ((
      SELECT ARRAY_AGG(DISTINCT TRIM(t))
      FROM UNNEST(val) t
      WHERE t != ""
    ));

    WITH source AS (
        SELECT ANY_VALUE(TitleDetails_title) AS title, TitleType AS type, 
            STRING_AGG(TitleDetails_LongSynopsis, " ") AS synopsis,
            LANGUAGE AS language,
            STRING_AGG(ARRAY_TO_STRING([TitleGenre, TitleSubgenres, TitleTags, LANGUAGE], ","), ",") AS keywords
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView`
        WHERE LANGUAGE IN ("eng", "spa")
        AND TitleType IN ("Programme", "Movie")
        AND TitleDetails_title NOT IN ("OnDemand Movie")
        GROUP BY LOWER(TitleDetails_title), TitleType, LANGUAGE
    ),

    test_data AS (
        SELECT title, type, language, synopsis, 
           strip_str_array(SPLIT(keywords, ",")) AS keywords
        FROM source cmv
        LEFT JOIN `res-nbcupea-dev-ds-sandbox-001.recsystem.ContentOrdinalId` cid
            ON LOWER(cmv.title) = LOWER(cid.program_title)
    )

    SELECT * FROM test_data

Downloading:  25%|██▌       | 44444/176668 [01:19<07:42, 285.92rows/s]

In [322]:
# mapping Centroid to dataset
query = """
    WITH orig AS (
        SELECT title AS program_title, type AS program_type, language AS program_language, 
            synopsis AS program_longsynopsis, tags
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_cmv_full_tags_edc_dev`
    ),
    
    
    mapper AS (
        SELECT tags, centroid_word, cluster
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.keyword_clusters`
    ),
    
    collections AS (
        SELECT a.program_title, a.program_type, a.program_language, a.program_longsynopsis,
        ARRAY_AGG(DISTINCT b.centroid_word) AS tags
        FROM orig a
        JOIN mapper b
        ON b.tags IN UNNEST(a.tags)
        GROUP BY a.program_title, a.program_type, a.program_language, a.program_longsynopsis
    )
    
    SELECT program_title, program_type, program_language, program_longsynopsis, tags
    FROM collections
"""

table_id = f"{client.project}.metadata_enhancement.synopsis_cmv_167_clustered_tags"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

client.query(query=query, job_config=job_config,  location="US").result()

In [324]:
%%bigquery
SELECT *
FROM metadata_enhancement.synopsis_cmv_167_clustered_tags
WHERE program_title = "The Office"

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.58s/rows]


,program_title,program_type,program_language,program_longsynopsis,tags
0,The Office,Programme,eng,Michael is overly confident about his team's c...,"[teens (ages 13-14), Christmas, Comedy, Sitcom..."


In [323]:
%%bigquery
SELECT DISTINCT t
FROM metadata_enhancement.synopsis_cmv_167_clustered_tags,
UNNEST(tags) t

Downloading: 100%|██████████| 165/165 [00:01<00:00, 121.97rows/s]


,t
0,Art
1,eng
2,Drama
3,Christmas
4,Comedy
...,...
160,Restaurant owner
161,Auction
162,Sitcom
163,Auto racing


In [11]:
# Full merlin dataset
query = """
    WITH movie_base AS (
        SELECT DISTINCT * FROM (
            SELECT program_type, program_val, program_title, program_longsynopsis, program_language,
                    tag_value
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    INNER JOIN `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.node2vec_token_edc_dev` n2v
                    ON n2v.tokens = tag_value
                    WHERE program_type LIKE 'Movie'
                    AND program_title NOT LIKE 'OnDemand Movie'
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL
                    AND program_seriesid IS NULL
                    

            UNION ALL
            SELECT DISTINCT
                    program_type,
                    program_val,
                    program_title,
                    program_longsynopsis,
                    program_language,
                    program_type as tag_value
            FROM content_metadata.merlin_tags tags
            INNER JOIN content_metadata.merlin_program progs
            ON tags.program_id = progs.program_val
            WHERE program_type LIKE 'Movie'
            AND program_title NOT LIKE 'OnDemand Movie'
            AND (program_language LIKE 'eng'
                        OR
                program_language LIKE 'spa')
            AND program_longsynopsis IS NOT NULL
            AND program_seriesid IS NULL

            UNION ALL
            SELECT DISTINCT
                    program_type,
                    program_val,
                    program_title,
                    program_longsynopsis,
                    program_language,
                    program_language as tag_value
            FROM content_metadata.merlin_tags tags
            INNER JOIN content_metadata.merlin_program progs
            ON tags.program_id = progs.program_val
            WHERE program_type LIKE 'Movie'
            AND program_title NOT LIKE 'OnDemand Movie'
            AND (program_language LIKE 'eng'
                        OR
                program_language LIKE 'spa')
            AND program_longsynopsis IS NOT NULL
            AND program_seriesid IS NULL

        )
    ),

    movie_final AS (
        SELECT
            program_type,
            program_title,
            program_longsynopsis,
            program_language,
            ARRAY_AGG(DISTINCT mb.tag_value) tags
        FROM movie_base mb
        GROUP BY
            program_type,
            program_title,
            program_longsynopsis,
            program_language
    ),
    series_base AS (
        SELECT DISTINCT * FROM (
            SELECT program_type, program_val, program_title, program_longsynopsis, 
                    program_language,
                    program_seriesid,
                    tag_Value,
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    INNER JOIN `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.node2vec_token_edc_dev` n2v
                    ON tag_value = n2v.tokens
                    WHERE (
                        program_type LIKE 'Episode'
                            OR program_type LIKE 'SeriesMaster'
                    )
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL

            UNION ALL
            SELECT program_type, program_val, program_title, program_longsynopsis, program_language,
                    program_seriesid, "Episode" AS tag_value
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    WHERE (
                        program_type LIKE 'Episode'
                            OR program_type LIKE 'SeriesMaster'
                    )
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL

            UNION ALL
            SELECT program_type, program_val, program_title, program_longsynopsis, program_language,
                    program_seriesid, program_language AS tag_value
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    WHERE (
                        program_type LIKE 'Episode'
                            OR program_type LIKE 'SeriesMaster'
                    )
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL

        )
    ),
    
    series_final AS (
        SELECT
            "Episode" AS program_type,
            program_title,
            STRING_AGG(DISTINCT program_longsynopsis, " ") AS program_longsynopsis,
            program_language,
            ARRAY_AGG(DISTINCT sb.tag_value) tags
        FROM series_base sb
        GROUP BY
            program_title,
            program_language
    )
    SELECT * FROM movie_final
    UNION ALL
    SELECT * FROM series_final
"""
table_id = f"{client.project}.metadata_enhancement.synopsis_full_tags_edc_dev"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

client.query(query=query, job_config=job_config,  location="US").result()

In [21]:
query = """
    WITH movie_base AS (
        SELECT DISTINCT * FROM (
            SELECT program_type, program_val, program_title, program_longsynopsis, program_language,
                    tag_value, 
                    CASE 
                        WHEN tag_type IN ("Genre", "KidsTheme") OR tag_value = "not for kids" THEN tag_value
                        ELSE CAST(NULL AS STRING)
                    END AS genre
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    INNER JOIN `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.node2vec_token_edc_dev` n2v
                    ON n2v.tokens = tag_value
                    WHERE program_type LIKE 'Movie'
                    AND program_title NOT LIKE 'OnDemand Movie'
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL
                    AND program_seriesid IS NULL

            UNION ALL
            SELECT DISTINCT
                    program_type,
                    program_val,
                    program_title,
                    program_longsynopsis,
                    program_language,
                    program_type AS tag_value,
                    CAST(NULL AS STRING) AS tag_type
            FROM content_metadata.merlin_tags tags
            INNER JOIN content_metadata.merlin_program progs
            ON tags.program_id = progs.program_val
            WHERE program_type LIKE 'Movie'
            AND program_title NOT LIKE 'OnDemand Movie'
            AND (program_language LIKE 'eng'
                        OR
                program_language LIKE 'spa')
            AND program_longsynopsis IS NOT NULL
            AND program_seriesid IS NULL

            UNION ALL
            SELECT DISTINCT
                    program_type,
                    program_val,
                    program_title,
                    program_longsynopsis,
                    program_language,
                    program_language as tag_value,
                    CAST(NULL AS STRING) AS tag_type
            FROM content_metadata.merlin_tags tags
            INNER JOIN content_metadata.merlin_program progs
            ON tags.program_id = progs.program_val
            WHERE program_type LIKE 'Movie'
            AND program_title NOT LIKE 'OnDemand Movie'
            AND (program_language LIKE 'eng'
                        OR
                program_language LIKE 'spa')
            AND program_longsynopsis IS NOT NULL
            AND program_seriesid IS NULL

        )
    ),

    movie_final AS (
        SELECT
            program_type,
            program_title,
            program_longsynopsis,
            program_language,
            ARRAY_AGG(DISTINCT genre IGNORE NULLS) AS genre,
            ARRAY_AGG(DISTINCT mb.tag_value) tags
        FROM movie_base mb
        GROUP BY
            program_type,
            program_title,
            program_longsynopsis,
            program_language
    ),
    series_base AS (
        SELECT DISTINCT * FROM (
            SELECT program_type, program_val, program_title, program_longsynopsis, 
                    program_language,
                    program_seriesid,
                    tag_value,
                    CASE 
                        WHEN tag_type IN ("Genre", "KidsTheme") OR tag_value = "not for kids" THEN tag_value
                        ELSE CAST(NULL AS STRING)
                    END AS genre
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    INNER JOIN `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.node2vec_token_edc_dev` n2v
                    ON tag_value = n2v.tokens
                    WHERE (
                        program_type LIKE 'Episode'
                            OR program_type LIKE 'SeriesMaster'
                    )
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL

            UNION ALL
            SELECT program_type, program_val, program_title, program_longsynopsis, program_language,
                    program_seriesid, "Episode" AS tag_value, CAST(NULL AS STRING) AS genre
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    WHERE (
                        program_type LIKE 'Episode'
                            OR program_type LIKE 'SeriesMaster'
                    )
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL

            UNION ALL
            SELECT program_type, program_val, program_title, program_longsynopsis, program_language,
                    program_seriesid, program_language AS tag_value, CAST(NULL AS STRING) AS genre
                    FROM content_metadata.merlin_tags tags
                    INNER JOIN content_metadata.merlin_program progs
                    ON tags.program_id = progs.program_val
                    WHERE (
                        program_type LIKE 'Episode'
                            OR program_type LIKE 'SeriesMaster'
                    )
                    AND (program_language LIKE 'eng'
                        OR
                        program_language LIKE 'spa')
                    AND program_longsynopsis IS NOT NULL

        )
    ),
    
    series_final AS (
        SELECT
            "Episode" AS program_type,
            program_title,
            STRING_AGG(DISTINCT program_longsynopsis, " ") AS program_longsynopsis,
            program_language,
            ARRAY_AGG(DISTINCT sb.genre IGNORE NULLS) AS genre,
            ARRAY_AGG(DISTINCT sb.tag_value) tags
        FROM series_base sb
        GROUP BY
            program_title,
            program_language
    )
    SELECT * FROM movie_final
    UNION ALL
    SELECT * FROM series_final
"""
table_id = f"{client.project}.metadata_enhancement.synopsis_full_tags_genre_edc_dev"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

client.query(query=query, job_config=job_config,  location="US").result()

# Clustering the keywords / tags

In [288]:
# Cluster the keywords
query = """
SELECT DISTINCT tags
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_full_tags_genre_edc_dev`,
UNNEST(tags) tags
"""

df_words = client.query(query=query, location="US").to_dataframe()
df_words

,tags
0,DJ
1,Music
2,Creative
3,California
4,Movie
...,...
5000,Chiméres
5001,President Aristide
5002,Port-au-Prince
5003,Seminola College


In [289]:
special_labels = ["eng", "spa", "Episode", "Movie", 'older teens (ages 15+)',
                'tweens (ages 10-12)',
                'teens (ages 13-14)',
                'big kids (ages 8-9)',
                'little kids (ages 5-7)',
                'preschoolers (ages 2-4)',
                'not for kids']
custom_kept_labels = ['Documentary', 'Comedy', 'Drama', 'Thriller', 'Horror', 'Romance',
       'Sitcom', 'Action & Adventure', 'Fantasy', 'Crime', 'Mystery',
       "Children's/Family Entertainment", 'History', 'Science fiction',
       'Sports', 'Educational', 'Animated', 'Military & War',
       'Western', 'Gay and Lesbian']

df_words = df_words.loc[~df_words["tags"].isin(special_labels)]
df_words

,tags
0,DJ
1,Music
2,Creative
3,California
5,Pursuit
...,...
5000,Chiméres
5001,President Aristide
5002,Port-au-Prince
5003,Seminola College


In [290]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

model = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")

dataset = tf.data.Dataset.from_tensor_slices(df_words["tags"].values).batch(50)
res = []
for batch in tqdm(dataset):
     res.append(model(batch))
df_words["embed"] = list(tf.concat(res, axis=0).numpy())
df_words.to_pickle("./scratch/hub_embed_keywords_embed2.pkl")
df_words



  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 675.41it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,tags,embed
0,DJ,"[-0.011037577, 0.06778954, -0.081553265, -0.04..."
1,Music,"[-0.057594366, 0.16248569, -0.12246627, 0.0655..."
2,Creative,"[-0.0642648, 0.20921557, -0.056612816, 0.06104..."
3,California,"[-0.00453534, 0.03185876, 0.19379067, -0.04166..."
5,Pursuit,"[-0.110649586, 0.09063735, 0.026033552, -0.002..."
...,...,...
5000,Chiméres,"[-0.20667349, -0.004094101, 0.026103506, 0.037..."
5001,President Aristide,"[-0.08246881, -0.0019505324, 0.032192685, 0.02..."
5002,Port-au-Prince,"[-0.12202282, 0.05105583, -0.09921123, 0.10520..."
5003,Seminola College,"[-0.1269239, 0.018460747, -0.020410243, 0.0417..."


In [291]:
## Clustering
from sklearn.cluster import KMeans
kmeans_clust = KMeans(n_clusters=140, random_state=42)
# Assign clusters
df_words["cluster"] = kmeans_clust.fit_predict(np.stack(df_words["embed"].values, axis=0))
df_words = df_words.sort_values("cluster").reset_index(drop=True)
# Figure out a "centeroid word"
# get the centroid
def centroid_word_func(pdf):
    values = np.stack(pdf["embed"].values, axis=0)
    centroid = values.mean(axis=0, keepdims=True)
    dist = np.sum((values - centroid)**2, axis=1)
    min_index = np.argmin(dist)
    pdf["centroid_word"] = pdf["tags"].iloc[min_index]
    pdf["num_words"] = pdf.shape[0]
    return pdf

df_words = df_words.groupby(by="cluster", as_index=False).progress_apply(centroid_word_func)
df_words

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 140/140 [00:00<00:00, 576.18it/s]


,tags,embed,cluster,centroid_word,num_words
0,Candy cane,"[-0.08472573, -0.14358519, 0.10053414, -0.0518...",0,Toy factory,11
1,Toy,"[-0.06649739, 0.09958365, 0.0536462, 0.0273038...",0,Toy factory,11
2,Toy factory,"[-0.048868295, 0.13744843, 0.12971462, 0.04136...",0,Toy factory,11
3,Toy maker,"[-0.053410415, 0.023874693, 0.012954429, -0.06...",0,Toy factory,11
4,Toys,"[-0.09476787, 0.15568548, -0.095866, 0.0462267...",0,Toy factory,11
...,...,...,...,...,...
4989,Affliction,"[-0.106540464, 0.015190207, -0.09322527, 0.031...",139,Punk,39
4990,Street punk,"[-0.082332194, 0.12206013, -0.004195919, 0.096...",139,Punk,39
4991,Punk rocker,"[-0.042677656, -0.020117873, -0.091137476, -0....",139,Punk,39
4992,Undertaker,"[-0.07260105, -0.020444859, -0.20298453, 0.053...",139,Punk,39


In [292]:
# Go through the custom list
for tt in custom_kept_labels:
    df_words[tt] = df_words["tags"].str.contains(tt[:4])
    
indices = [np.where(ii)[0][0]  if len(np.where(ii)[0]) > 0 else -1 for ii in df_words.iloc[:, 5:].values]
for jj, ii in enumerate(indices):
    if ii < 0: continue
    df_words.loc[jj, "centroid_word"] = custom_kept_labels[ii]
df_words

,tags,embed,cluster,centroid_word,num_words,Documentary,Comedy,Drama,Thriller,Horror,...,Mystery,Children's/Family Entertainment,History,Science fiction,Sports,Educational,Animated,Military & War,Western,Gay and Lesbian
0,Candy cane,"[-0.08472573, -0.14358519, 0.10053414, -0.0518...",0,Toy factory,11,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Toy,"[-0.06649739, 0.09958365, 0.0536462, 0.0273038...",0,Toy factory,11,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Toy factory,"[-0.048868295, 0.13744843, 0.12971462, 0.04136...",0,Toy factory,11,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Toy maker,"[-0.053410415, 0.023874693, 0.012954429, -0.06...",0,Toy factory,11,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Toys,"[-0.09476787, 0.15568548, -0.095866, 0.0462267...",0,Toy factory,11,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4989,Affliction,"[-0.106540464, 0.015190207, -0.09322527, 0.031...",139,Punk,39,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4990,Street punk,"[-0.082332194, 0.12206013, -0.004195919, 0.096...",139,Punk,39,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4991,Punk rocker,"[-0.042677656, -0.020117873, -0.091137476, -0....",139,Punk,39,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4992,Undertaker,"[-0.07260105, -0.020444859, -0.20298453, 0.053...",139,Punk,39,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [293]:
df_words = df_words.drop(columns=custom_kept_labels)
df_words

,tags,embed,cluster,centroid_word,num_words
0,Candy cane,"[-0.08472573, -0.14358519, 0.10053414, -0.0518...",0,Toy factory,11
1,Toy,"[-0.06649739, 0.09958365, 0.0536462, 0.0273038...",0,Toy factory,11
2,Toy factory,"[-0.048868295, 0.13744843, 0.12971462, 0.04136...",0,Toy factory,11
3,Toy maker,"[-0.053410415, 0.023874693, 0.012954429, -0.06...",0,Toy factory,11
4,Toys,"[-0.09476787, 0.15568548, -0.095866, 0.0462267...",0,Toy factory,11
...,...,...,...,...,...
4989,Affliction,"[-0.106540464, 0.015190207, -0.09322527, 0.031...",139,Punk,39
4990,Street punk,"[-0.082332194, 0.12206013, -0.004195919, 0.096...",139,Punk,39
4991,Punk rocker,"[-0.042677656, -0.020117873, -0.091137476, -0....",139,Punk,39
4992,Undertaker,"[-0.07260105, -0.020444859, -0.20298453, 0.053...",139,Punk,39


In [294]:
# Adding the special words
df_special = pd.DataFrame({"tags": [kk for kk in special_labels], 
                           "embed" : [[]] * len(special_labels),
                           "cluster": np.arange(140, 140+len(special_labels)),
                           "centroid_word": [kk for kk in special_labels],
                           "num_words": np.ones(len(special_labels), dtype=int)}
                          )
df_final_map = df_words.append(df_special)
df_final_map



,tags,embed,cluster,centroid_word,num_words
0,Candy cane,"[-0.08472573, -0.14358519, 0.10053414, -0.0518...",0,Toy factory,11
1,Toy,"[-0.06649739, 0.09958365, 0.0536462, 0.0273038...",0,Toy factory,11
2,Toy factory,"[-0.048868295, 0.13744843, 0.12971462, 0.04136...",0,Toy factory,11
3,Toy maker,"[-0.053410415, 0.023874693, 0.012954429, -0.06...",0,Toy factory,11
4,Toys,"[-0.09476787, 0.15568548, -0.095866, 0.0462267...",0,Toy factory,11
...,...,...,...,...,...
6,teens (ages 13-14),[],146,teens (ages 13-14),1
7,big kids (ages 8-9),[],147,big kids (ages 8-9),1
8,little kids (ages 5-7),[],148,little kids (ages 5-7),1
9,preschoolers (ages 2-4),[],149,preschoolers (ages 2-4),1


In [295]:
df_final_map["centroid_word"].unique().shape

(167,)

In [296]:
df_final_map.to_gbq("metadata_enhancement.keyword_clusters",
    project_id=client.project,
    if_exists='replace')

1it [00:11, 11.49s/it]


# Mapping centroid words to dataset

In [25]:
query = """
    WITH orig AS (
        SELECT program_title, program_type, program_language, program_longsynopsis, genre, tags
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_full_tags_genre_edc_dev`
    ),
    
    
    mapper AS (
        SELECT tags, centroid_word, cluster
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.keyword_clusters`
    ),
    
    collections AS (
        SELECT a.program_title, a.program_type, a.program_language, a.program_longsynopsis,
        ANY_VALUE(a.genre) AS genre,
        ANY_VALUE(a.tags) AS keywords,
        ARRAY_AGG(DISTINCT b.centroid_word) AS tags
        FROM orig a
        JOIN mapper b
        ON b.tags IN UNNEST(a.tags)
        GROUP BY a.program_title, a.program_type, a.program_language, a.program_longsynopsis
    )
    
    SELECT program_title, program_type, program_language, program_longsynopsis, genre, keywords, tags
    FROM collections
    WHERE program_title IS NOT NULL 
        AND program_type IS NOT NULL
        AND program_longsynopsis IS NOT NULL
        AND ARRAY_LENGTH(tags) > 0
"""

table_id = f"{client.project}.metadata_enhancement.synopsis_genres_167_clustered_tags"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

client.query(query=query, job_config=job_config, location="US").result()


In [311]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding",
    "Un adelanto y los comentarios de Kevin Costner y el elenco del programa. Un secreto sobre John sale a la luz; Beth le muestra a Jenkins una noche difícil; un tiempo especial con Tate conduce a casi un desastre. Con el calor del verano la naturaleza de Yellowstone se ha mostrado en toda su plenitud, en tan solo unos pocos meses ha florecido de manera extraordinaria. Pero ahora en tan solo unas semanas volverá la nieve y el frío."
])

print(embeddings)

tf.Tensor(
[[-0.07796571 -0.07574748  0.02141027 ... -0.01937356  0.05496128
   0.00755278]
 [-0.06348769 -0.03529344  0.03835971 ... -0.00586085 -0.01329273
   0.10802048]
 [ 0.06036134  0.01039356 -0.07722514 ... -0.06877957  0.05509286
  -0.03794941]], shape=(3, 512), dtype=float32)


In [312]:
summ = 0
for ii in embed.variables:
    summ += (np.prod(ii.shape))
print(summ)



68927232


# How does raw content to content look?

In [330]:
%%bigquery tag_index

SELECT DISTINCT tags
FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_cmv_167_clustered_tags`,
UNNEST(tags) tags

Downloading: 100%|██████████| 165/165 [00:01<00:00, 109.85rows/s]


In [335]:
tag_list = tag_index["tags"].tolist()
tag_list

['Art',
 'eng',
 'Drama',
 'Christmas',
 'Comedy',
 'Romance',
 'Army',
 'Car',
 'Crime',
 'Race',
 'Action & Adventure',
 'Mountain',
 'Spirit',
 'Comedian',
 'Animal',
 'Soldiers',
 'Grandmother',
 'Murder',
 'Bounty hunter',
 'American South',
 'Pervert',
 'Su-Su',
 'Thrilling',
 'Thriller',
 'Mystery',
 'Extortion',
 'Zombies',
 'Warrior',
 'Austria',
 'older teens (ages 15+)',
 'Dracula',
 'Confusion',
 'Running away',
 '5th century',
 'City',
 'Reality',
 'Prison',
 'Aviation',
 'History',
 'War',
 'Military & War',
 'Poetry',
 'teens (ages 13-14)',
 'Sexy woman',
 'Russian mob',
 'Family',
 '1870s',
 "Children's/Family Entertainment",
 'White House',
 'New Zealand',
 'Adolescent boy',
 'Fantasy',
 'Political turmoil',
 'Emperor',
 'Equestrian',
 'Science',
 'Richard Nixon',
 'John Holmes',
 'Death of parent',
 'Missouri',
 'Professionalism',
 'Action figure',
 'Racism',
 'Family member',
 'Road',
 'Chicago',
 'Amusement park',
 'Computer',
 'Lounge',
 'Christianity',
 'Mother/da

In [9]:
%%bigquery unscored_titles

WITH cid AS (
    SELECT DISTINCT program_title, content_ordinal_id
    FROM `res-nbcupea-dev-ds-sandbox-001.recsystem.ContentOrdinalId`
)

SELECT a.program_title, a.program_type, a.program_longsynopsis, a.program_language, 
    STRING_AGG(DISTINCT t, " ") AS keywords, b.content_ordinal_id
FROM `metadata_enhancement.synopsis_cmv_167_clustered_tags` a,
UNNEST(a.tags) t
JOIN cid b
ON LOWER(a.program_title) = LOWER(b.program_title)
GROUP BY a.program_title, a.program_type, a.program_language, a.program_longsynopsis, b.content_ordinal_id


Downloading: 100%|██████████| 2286/2286 [00:02<00:00, 952.35rows/s] 


In [341]:
def tags_to_index(tags, tags_list):
    col = np.array([tags_list.index(t) for t in tags], dtype=int)
    data = np.ones(len(col), dtype=int)
    row = np.zeros(len(col), dtype=int)
    return coo_matrix((data, (row, col)), shape=(1, len(tags_list))).A[0]

df_test["labels"] = df_test["tags"].progress_apply(lambda x: tags_to_index(x, tag_list))
df_test

100%|██████████| 2286/2286 [00:00<00:00, 11166.91it/s]


,program_title,program_type,program_longsynopsis,program_language,tags,content_ordinal_id,labels
0,Christmas Matchmakers,Movie,Two overworked personal assistants hatch a pla...,eng,"[Art, eng, Drama, Christmas, Comedy, Romance]",1678,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,John Wick 2,Movie,Legendary hit man John Wick comes out of retir...,eng,"[Art, Army, Car, Crime, Race, eng, Action & Ad...",2270,"[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Ashes in the Snow,Movie,"During World War II, a 16-year-old artist and ...",eng,"[Art, Comedy, Austria, Reality, Romance, Priso...",111,"[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Alice in Wonderland,Movie,Little Alice (Charlotte Henry) meets the Chesh...,eng,"[Art, Aviation, 1870s, Children's/Family Enter...",62,"[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ..."
4,Thomas Kinkade's Christmas Cottage,Movie,"Inspired by his mentor (Peter O'Toole), a youn...",eng,"[Art, Adolescent boy, Death of parent, Mountai...",954,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ..."
...,...,...,...,...,...,...,...
2281,Fievel's American Tails,Programme,Fievel and the Mousekewitz family. Animated. F...,eng,"[Children's/Family Entertainment, Adolescent b...",348,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2282,DreamWorks Nursery Rhymes,Programme,"You'll feel great moving around to ""Ready To G...",eng,"[Children's/Family Entertainment, Adolescent b...",306,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2283,Baby Einstein,Programme,A children's safari that encourages vocal part...,eng,"[Children's/Family Entertainment, Adolescent b...",120,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2284,Stone Age,Programme,"It's not Billy's day today, and with everythin...",eng,"[Children's/Family Entertainment, Adolescent b...",1142,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


In [345]:
def cosine_sim(P):
    P = P / np.sqrt(np.sum(P**2, axis=1, keepdims=True))
    cos_sim_c2c = P @ P.T
    cos_sim_c2c = np.nan_to_num(cos_sim_c2c, nan=-1)
    return cos_sim_c2c

labels = np.stack(df_test["labels"].values, axis=0)
similarity = cosine_sim(labels)

# Slice out top 15 recommendations
score = list(np.sort(similarity, axis=1)[:, ::-1][:, 1:(15+1)])
sim_c2c_argsort = np.argsort(similarity, axis=1)[:, ::-1][:, 1:]
titles = list(np.take(df_test["program_title"].values, sim_c2c_argsort[:, :15]))
titles_type = list(np.take(df_test["program_type"].values, sim_c2c_argsort[:, :15]))
synopsis = list(np.take(df_test["program_longsynopsis"].values, sim_c2c_argsort[:, :15]))
dict_list = [{"program_title": tt, "program_type": ttype, "program_longsynopsis": syn,  "score": sc} \
             for tt, ttype, syn, sc in zip(titles, titles_type, synopsis, score)]

df_test["top15"] = dict_list
# Calculate a type match
df_test["type_match"] = (np.stack(titles_type) == df_test["program_type"][:, None]).mean(axis=1)

df_test

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


,program_title,program_type,program_longsynopsis,program_language,tags,content_ordinal_id,labels,top15,type_match
0,Christmas Matchmakers,Movie,Two overworked personal assistants hatch a pla...,eng,"[Art, eng, Drama, Christmas, Comedy, Romance]",1678,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'program_title': ['A Christmas Princess', 'A ...",1.000000
1,John Wick 2,Movie,Legendary hit man John Wick comes out of retir...,eng,"[Art, Army, Car, Crime, Race, eng, Action & Ad...",2270,"[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{'program_title': ['Precious Cargo', 'John Wic...",1.000000
2,Ashes in the Snow,Movie,"During World War II, a 16-year-old artist and ...",eng,"[Art, Comedy, Austria, Reality, Romance, Priso...",111,"[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'program_title': ['Schindler's List', 'Cesar ...",1.000000
3,Alice in Wonderland,Movie,Little Alice (Charlotte Henry) meets the Chesh...,eng,"[Art, Aviation, 1870s, Children's/Family Enter...",62,"[1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...","{'program_title': ['Essential Killing', 'Curio...",1.000000
4,Thomas Kinkade's Christmas Cottage,Movie,"Inspired by his mentor (Peter O'Toole), a youn...",eng,"[Art, Adolescent boy, Death of parent, Mountai...",954,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...",{'program_title': ['Girl With a Pearl Earring'...,1.000000
...,...,...,...,...,...,...,...,...,...
2281,Fievel's American Tails,Programme,Fievel and the Mousekewitz family. Animated. F...,eng,"[Children's/Family Entertainment, Adolescent b...",348,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","{'program_title': ['My Life Me', 'Fievel's Ame...",0.866667
2282,DreamWorks Nursery Rhymes,Programme,"You'll feel great moving around to ""Ready To G...",eng,"[Children's/Family Entertainment, Adolescent b...",306,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'program_title': ['Timmy the Tooth', 'Blue Ba...",0.866667
2283,Baby Einstein,Programme,A children's safari that encourages vocal part...,eng,"[Children's/Family Entertainment, Adolescent b...",120,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{'program_title': ['Little Baby Bum', 'Guess W...",0.933333
2284,Stone Age,Programme,"It's not Billy's day today, and with everythin...",eng,"[Children's/Family Entertainment, Adolescent b...",1142,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","{'program_title': ['My Life Me', 'Fievel's Ame...",0.866667


In [346]:
from IPython.display import display
def query_shows_c2c(df_titles, show_name, show_res=True):
    pdf = df_titles.loc[df_titles["program_title"]==show_name, :]
    pdf_query = pdf[["program_title", "program_type", "program_longsynopsis"]]
    pdf_res = pd.DataFrame(pdf[f"top15"].values[0])
    if show_res:
        print("Query")
        display(pdf_query)
        display(pdf_res)
    
    return pdf_query, pdf_res


In [348]:
important_titles = ["The Office", "30 Rock", "Punky Brewster", "Parks and Recreation", "WWE Monday Night RAW", 
            "Yellowstone", "Saturday Night Live", "Law & Order: Special Victims Unit", 
            "Mr. Mercedes", "Happy Feet Two", "Zombie Tidal Wave"]

for ti in important_titles:
    _ = query_shows_c2c(df_test, ti)

Query


,program_title,program_type,program_longsynopsis
2250,The Office,Programme,Michael is overly confident about his team's c...


,program_title,program_type,program_longsynopsis,score
0,Grace Under Fire,Programme,Grace's Valentine's Day date with suitor Ryan ...,1.000000
1,Everybody Loves Raymond,Programme,"After having a fight with Frank, Marie moves i...",1.000000
2,The Jeff Foxworthy Show,Programme,A fishing trip with Karen and two friends goes...,1.000000
3,George Lopez,Programme,George and Benny try to fill jailed Jim's spot...,0.912871
4,Grounded for Life,Programme,"When Sean receives a small inheritance, he mus...",0.912871
5,Perfect Harmony,Programme,When Reverend Jax is visited by his charismati...,0.912871
6,Odd Mom Out,Programme,Jill falls in love with the down-to-earth moms...,0.894427
7,Dennis the Menace,Programme,The widowed Grandpa Perkins visits but resists...,0.894427
8,Roseanne,Programme,Dan's 15-year-old secret is exposed during pre...,0.845154
9,The Goode Family,Programme,Gerald tries to inspire his children by adopti...,0.800000


Query


,program_title,program_type,program_longsynopsis
2046,30 Rock,Programme,Jack meets with German TV executives to close ...


...

,program_title,program_type,program_longsynopsis
234,Happy Feet Two,Movie,"Reluctant to dance, the son of Mumble (Elijah ..."


,program_title,program_type,program_longsynopsis,score
0,The Land Before Time XIV: Journey of the Brave,Movie,Littlefoot goes on a journey to find his fathe...,0.734847
1,Happy Feet,Movie,"Born without the ability to sing, a young empe...",0.692820
2,The Land Before Time: The Wisdom of Friends,Movie,Littlefoot and friends help two clumsy dinosau...,0.642364
3,Trolls World Tour,Movie,When a rock 'n' roll king and queen set out to...,0.639602
4,The Land Before Time,Movie,An orphaned dinosaur and his new friends face ...,0.577350
5,Golden Winter,Movie,A boy helps abandoned golden retriever puppies...,0.571548
6,The Jungle Bunch,Movie,A penguin raised by tigers defends his jungle ...,0.560000
7,Mamma Mia!,Movie,Hoping to meet her real father and have him wa...,0.557086
8,"Curious George: Go West, Go Wild",Movie,"While farm-sitting, Ted goes fishing and Georg...",0.554322
9,Amanda and Jack Go Glamping,Movie,With his marriage and career against the ropes...,0.550598


Query


,program_title,program_type,program_longsynopsis
435,Zombie Tidal Wave,Movie,Zombies wreak bloodthirsty havoc after a tidal...


,program_title,program_type,program_longsynopsis,score
0,Zombie Tidal Wave,Movie,Zombies wreak bloodthirsty havoc after a tidal...,1.000000
1,Killer High,Movie,Sabrina's perfectly planned high school reunio...,0.925820
2,Neverknock,Movie,After Grace and her friends ignore the warning...,0.857143
3,Campus Code,Movie,When one of their classmates disintegrates rig...,0.857143
4,The Mummy's Curse,Movie,"Shipped to Louisiana, mummy Kharis (Lon Chaney...",0.845154
5,The Invisible Man's Revenge,Movie,A criminal (Jon Hall) haunts his enemies' mans...,0.845154
6,The Invisible Woman,Movie,A screwy professor's experiments with a gorgeo...,0.845154
7,Scorched Earth,Movie,A post-apocalyptic bounty hunter tries to brin...,0.845154
8,Body Bags,Movie,Filmmaker John Carpenter introduces a trio of ...,0.845154
9,Android Cop,Movie,A cop and his android partner enter a forbidde...,0.845154
